In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Codes\\Codes\\Python\\Summlytics'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from Summlytics.constants import *
from Summlytics.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config

In [6]:
import urllib.request as request
import zipfile
from Summlytics.logging import logger
from Summlytics.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_data(self):
        logger.info("Downloading data")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_url, self.config.local_data_file)
            logger.info(f"{filename} downlaoded with the following information: \n{headers}")
        else:
            logger.info(f"Data file already exists at {self.config.local_data_file} of size: {get_size(Path(self.config.local_data_file))}")


    def unzip_data(self):
        logger.info("Unzipping data")

        os.makedirs(self.config.unzip_dir, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Unzipped data to {self.config.unzip_dir}")

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    error_message = f"Error occurred while downloading and unzipping the data: {str(e)}"
    logger.error(error_message, exc_info=True)
    raise ValueError(error_message) from e

[2024-02-07 18:24:20,910: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully.]
[2024-02-07 18:24:20,911: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='cp1252'> loaded successfully.]
[2024-02-07 18:24:20,913: INFO: common: Directory: artifacts created.]
[2024-02-07 18:24:20,914: INFO: common: Directory: artifacts/data_ingestion created.]
[2024-02-07 18:24:20,914: INFO: 3860852999: Downloading data]
[2024-02-07 18:24:22,551: INFO: 3860852999: artifacts/data_ingestion/data.zip downlaoded with the following information: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mo